In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, DataCollatorWithPadding
import pandas as pd
import numpy as np
from datasets import Dataset, DatasetDict
from torch.utils.data import DataLoader
import json

In [8]:
with open('./party_lst.json') as json_file:
    parties = json.load(json_file)
parties

['Socialdemokratiet',
 'Venstre',
 'Socialistisk Folkeparti',
 'Det Radikale Venstre',
 'Enhedslisten',
 'Det Konservative Folkeparti',
 'Dansk Folkeparti',
 'Nye Borgerlige',
 'Frie Grønne, Danmarks Nye Venstrefløjsparti',
 'Liberal Alliance',
 'Alternativet',
 'Inuit Ataqatigiit',
 'Javnaðarflokkurin',
 'Kristendemokraterne',
 'Siumut',
 'Sambandsflokkurin',
 'Uden for folketingsgrupperne',
 'Nunatta Qitornai',
 'Tjóðveldi',
 'Tjóðveldisflokkurin',
 'Ny Alliance',
 'Fólkaflokkurin']

## Flow for each party

In [ ]:
party = 

In [ ]:
df_train = pd.read_csv("./by_party_train" + party + ".csv")
df_test = pd.read_csv("./by_party_test" + party + ".csv")